In [ ]:
import os
import glob
import time
import requests
import pandas as pd
from datetime import datetime
from statistics import mean
import xlwt
from tqdm import tqdm

session = requests.Session()

geocode_cache = {}
weather_cache = {}
geocode_df = None

def geocode_city(city, country=None):
    global geocode_cache, geocode_df
    if not city or pd.isna(city):
        return (None, None)
    key = (str(city).strip().lower(), (str(country).strip().lower() if country else None))
    if key in geocode_cache:
        return geocode_cache[key]

    geocode_df = pd.read_excel(os.path.join(os.getcwd(), "data", "worldcities.xlsx"))

    lookup = str(city).strip().lower()
    city_col = geocode_df.get("city", pd.Series(dtype="str")).astype(str).str.lower()
    ascii_col = geocode_df.get("city_ascii", pd.Series(dtype="str")).astype(str).str.lower()
    mask = (city_col == lookup) | (ascii_col == lookup)
    if country:
        country_col = geocode_df.get("country", pd.Series(dtype="str")).astype(str).str.lower()
        mask = mask & (country_col == str(country).strip().lower())

    candidates = geocode_df[mask]
    if candidates.empty:
        geocode_cache[key] = (None, None)
        return (None, None)

    if "population" in candidates.columns:
        candidates = candidates.copy()
        candidates["population"] = pd.to_numeric(candidates["population"], errors="coerce").fillna(0)
        row = candidates.sort_values("population", ascending=False).iloc[0]
    else:
        row = candidates.iloc[0]

    lat = float(row.get("lat")) if not pd.isna(row.get("lat")) else None
    lon = float(row.get("lng")) if not pd.isna(row.get("lng")) else None
    geocode_cache[key] = (lat, lon)
    return (lat, lon)



def fetch_weather_mean(lat, lon, date_obj):
    if lat is None or lon is None:
        return (None, None)

    date_str = pd.to_datetime(date_obj, errors="coerce")
    if pd.isna(date_str):
        return (None, None)
    date_str = date_str.strftime("%Y-%m-%d")

    key = (round(lat, 4), round(lon, 4), date_str)
    if key in weather_cache:
        return weather_cache[key]

    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": date_str,
        "end_date": date_str,
        "hourly": "temperature_2m,relativehumidity_2m",
        "timezone": "auto",
    }

    try:
        r = session.get("https://archive-api.open-meteo.com/v1/archive", params=params, timeout=20)
        r.raise_for_status()
    except requests.HTTPError as e:
        print("HTTP error fetching weather:", e, "response:", getattr(e.response, "text", None))
        return (None, None)
    except requests.RequestException as e:
        print("Request exception fetching weather:", e)
        return (None, None)

    data = r.json()
    hourly = data.get("hourly", {})
    times = hourly.get("time", [])
    temps = hourly.get("temperature_2m", [])
    hums = hourly.get("relativehumidity_2m", [])

    daytime_temps = []
    daytime_hums = []
    for t_str, tp, hm in zip(times, temps, hums):
        try:
            dt = pd.to_datetime(t_str)
        except Exception:
            continue
        if 6 <= dt.hour <= 18:
            daytime_temps.append(tp)
            daytime_hums.append(hm)

    mean_temp = float(mean(daytime_temps)) if daytime_temps else None
    mean_hum = float(mean(daytime_hums)) if daytime_hums else None

    time.sleep(0.5)
    weather_cache[key] = (mean_temp, mean_hum)
    return (mean_temp, mean_hum)






In [ ]:

path = "./data/"
def main(year:str):
    input_xls = path + year + ".xls"
    df = pd.read_excel(input_xls)

    # parse dates like 1/1/2001 -> 2001-01-01
    df["Date_parsed"] = pd.to_datetime(df["Date"], dayfirst=False, errors="coerce")

    # prepare result columns
    df["temp_matchday"] = None
    df["humidity_matchday"] = None

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows for "+input_xls):
        court = row.get("Court")
        if court == "Outdoor":
            city = row.get("Location")
            date_obj = row.get("Date_parsed")
            lat, lon = geocode_city(city)
            temp, hum = fetch_weather_mean(lat, lon, date_obj)
            df.at[idx, "temp_matchday"] = temp
            df.at[idx, "humidity_matchday"] = hum

    out_xls = os.path.join(path, year + ".csv")
    df.to_csv(out_xls, index=False)
    print("Saved", out_xls)



years = ["2010", "2011", "2012", "2013", "2014", "2015", "2016", "2018","2019", "2020", "2021", "2022", "2023", "2024", "2025"]
for y in years:
    main(y)

Processing rows for ./data/2010.xls: 100%|██████████| 2679/2679 [02:59<00:00, 14.93it/s]


Saved ./data/2010.csv


Processing rows for ./data/2011.xls: 100%|██████████| 2675/2675 [02:53<00:00, 15.42it/s]


Saved ./data/2011.csv


Processing rows for ./data/2012.xls:  12%|█▏        | 318/2607 [00:27<03:14, 11.76it/s]